# 05 | Deployment - LoanVet Credit Risk Model

This notebook demonstrates how to deploy the final XGBoost credit risk classification model for LoanVet. The deployment pipeline includes:

- Loading the trained model and classification threshold
- Defining batch and single-record prediction functions
- Serving the model via a simple API (using FastAPI)
- Logging, evaluation sanity checks, and downstream app integration
- Documenting deployment considerations and best practices

This approach ensures a reproducible, auditable, and production-ready workflow, especially important for financial analytics applications.

In [ ]:
import os
import json
import joblib
import logging
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from typing import Union, List, Dict
from fastapi import FastAPI, HTTPException
from pydantic import RootModel

# Configure logging to output INFO level messages to console
logging.basicConfig(level=logging.INFO)

## Load Final Model & Metadata

We load the trained XGBoost model and metadata including the classification threshold and feature list.  
This metadata ensures that incoming data features match the model expectations exactly.

In [ ]:
MODEL_PATH = '../models/final/xgb_final_model.joblib'
METADATA_PATH = '../models/final/xgb_final_metadata.json'

# Load model
try:
    model: XGBClassifier = joblib.load(MODEL_PATH)
    logging.info("Model loaded from models/final/xgb_final_model.joblib.")
    print("Model loaded.")
except Exception as e:
    logging.error(f"Error loading model: {e}")
    raise RuntimeError("Failed to load model.")

# Load metadata
try:
    with open(METADATA_PATH, 'r') as f:
        metadata = json.load(f)
    
    threshold = metadata.get("threshold", 0.5)  # fallback threshold
    feature_list = metadata.get("features", [])
    
    logging.info(f"Threshold loaded: {threshold}")
    logging.info(f"Feature list loaded with {len(feature_list)} features.")
    
    print(f"Threshold loaded: {threshold}")
    print(f"Features loaded: {len(feature_list)} features")
except Exception as e:
    logging.error(f"Error loading metadata: {e}")
    raise RuntimeError("Failed to load metadata.")

Model loaded.
Threshold loaded: 0.2268
Features loaded: 19 features


## Define Prediction Functions

These functions encapsulate the prediction logic:

- `predict_single`: For a single record passed as a dictionary, predict the probability and risk label using the threshold.
- `predict_batch`: For a pandas DataFrame of multiple records, predict probabilities and labels for all.

Separating these functions allows modular reuse in batch processing, APIs, or interactive apps.

In [ ]:
def predict_single(record: Dict[str, Union[float, int]]) -> Dict[str, Union[int, float]]:
    try:
        # Convert input dict to DataFrame with one row
        df = pd.DataFrame([record])

        # Ensure columns are in correct order and all features present
        df = df[feature_list]

        # Model predicts probability of positive class (1)
        proba = model.predict_proba(df)[:, 1][0]

        # Apply classification threshold
        label = int(proba >= threshold)

        return {"label": label, "probability": proba}

    except Exception as e:
        logging.error(f"Error in single prediction: {e}")
        raise

def predict_batch(df: pd.DataFrame) -> pd.DataFrame:
    try:
        # Ensure columns are in correct order and all features present
        df = df[feature_list]

        # Predict probabilities for positive class
        proba = model.predict_proba(df)[:, 1]

        # Apply threshold
        labels = (proba >= threshold).astype(int)

        # Append results
        df_result = df.copy()
        df_result["probability"] = proba
        df_result["label"] = labels

        return df_result

    except Exception as e:
        logging.error(f"Error in batch prediction: {e}")
        raise

## Build FastAPI Application and Define /predict Endpoint

We create a FastAPI app that exposes an HTTP POST `/predict` endpoint.  
The endpoint:

- Accepts JSON input with feature values for a single record
- Validates input features using Pydantic models
- Runs prediction with `predict_single()`
- Returns the predicted label and probability as JSON

This API supports real-time inference and easy integration with downstream applications such as a Streamlit frontend.

In [ ]:
# Pydantic model for input validation
class PredictionRequest(RootModel):
    pass

app = FastAPI(title="LoanVet Credit Risk Model API")

@app.post("/predict")
async def predict_endpoint(input_data: PredictionRequest):
    input_dict = input_data.model_dump()
    
    missing_features = set(feature_list) - set(input_dict.keys())
    extra_features = set(input_dict.keys()) - set(feature_list)

    if missing_features:
        raise HTTPException(
            status_code=422,
            detail=f"Missing required features: {missing_features}"
        )
    if extra_features:
        raise HTTPException(
            status_code=422,
            detail=f"Unexpected extra features provided: {extra_features}"
        )

    result = predict_single(input_dict)
    logging.info(f"Prediction made: {result}")
    return {"prediction": result}

## Integrating with the Streamlit Frontend

The deployed FastAPI backend exposes a `/predict` endpoint that accepts JSON requests containing feature values and returns the credit risk prediction results. 

To provide an interactive user interface for end users, we deploy a separate **Streamlit app** that acts as the frontend. This app:

- Collects user inputs for all required features via input widgets.
- Sends the input data as a JSON payload in a POST request to the FastAPI `/predict` endpoint.
- Displays the returned prediction label ("High Risk" or "Low Risk") and the associated probability.

### Deployment Workflow

1. **Backend API Deployment:**
   - The FastAPI application is containerized and deployed to a cloud platform (e.g., Railway, Heroku, or AWS).
   - The deployed API endpoint URL is made publicly accessible (e.g., `https://loanvet-api.up.railway.app/predict`).

2. **Frontend Deployment:**
   - The Streamlit app is deployed independently on Streamlit Community Cloud.
   - In the Streamlit app code, the API URL is configured to point to the deployed backend endpoint.
   - This separation allows frontend and backend to scale and update independently.

This end-to-end deployment pipeline provides a scalable, modular, and user-friendly system for credit risk prediction in financial analytics applications.